In [1]:
import pandas as pd
import torch
import matplotlib.pyplot as plt
PATH = '/home/arsentii/prog/camda/'
import numpy as np
import torch
import re
import spacy
from collections import defaultdict


from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import nltk
spacy.prefer_gpu()
nlp = spacy.load("en_core_sci_md", disable=['ner', 'parser'])
device = torch.device('cpu')

# Plan
Entry point classify_pmi_based
+ 